Librairies

In [3]:
import os
import numpy as np
import pandas as pd
import cv2

from matplotlib import pyplot as plt
from scipy.stats import pearsonr
from skimage.metrics import structural_similarity as ssim

Metriques

In [4]:
# Ouvrir les images
def img_open(path_sal_map,path_attacked_sal_map):
    img1 = cv2.imread(path_sal_map)
    img2 = cv2.imread(path_attacked_sal_map)
    return img1,img2

# Calcul de la différence absolue
# Plus la valeur est proche de 0 plus les images sont similaires
def diff_abs(path_sal_map,path_attacked_sal_map):
    img1 = cv2.imread(path_sal_map)
    img2 = cv2.imread(path_attacked_sal_map)
    return (np.sum(np.abs(img1 - img2)))/(img1.size)

# Calcul de la différence quadratique (mse)
# Plus la valeur est proche de 0 plus les images sont similaires
def diff_quadratique(path_sal_map,path_attacked_sal_map):
    img1, img2 = img_open(path_sal_map,path_attacked_sal_map)
    diff = cv2.subtract(img1, img2)
    error = np.sum(np.square(diff))
    return error/(img1.size)


# Calcul du coef de corrélation
# Plus la valeur est proche de 1 plus les images sont similaires
def coef_correlation(path_sal_map,path_attacked_sal_map):
    img1, img2 = img_open(path_sal_map,path_attacked_sal_map)
    result = pearsonr(img1.flatten(),img2.flatten())
    return result.statistic

# Calcul du ssim Structural Similarity Index
# Plus la valeur est proche de 1 plus les images sont similaires
def ssim_func(path_sal_map,path_attacked_sal_map):
    img1, img2 = img_open(path_sal_map,path_attacked_sal_map)
    gray_img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray_img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    valssim = ssim(gray_img1, gray_img2)
    return valssim

In [14]:
path_project_root = os.path.dirname(os.path.abspath(''))
directory = os.path.join(path_project_root, 'final_dir\images\output')

obj = []
model = []
attack = []
intensity= []
saliency = []
attacked = []
path = []

for file in os.listdir(directory):
    filepath = os.path.join(directory, file)
    path.append(filepath)

    filename = file[:-4]
    info = filename.split('_')
    obj.append(info[0])
    model.append(info[1])
    attack.append(info[2])
    intensity.append(info[3])
    saliency.append(info[4])
    attacked.append(info[5])
    
  
df = pd.DataFrame(list(zip(obj,model,attack,intensity,saliency,attacked,path)), 
                  columns = ['Objet','Model','Attaque','Intensité','Saillance','Attaquée','Chemin'])


df_saliency_attacked = df[(df['Saillance'] == 'saliency') & (df['Attaquée'] == 'attacked')].reset_index()
df_saliency_noattacked = df[(df['Saillance'] == 'saliency') & (df['Attaquée'] == 'noattacked')].reset_index()

df_img_attacked = df[(df['Saillance'] == 'nosaliency') & (df['Attaquée'] == 'attacked')].reset_index()
df_img_noattacked = df[(df['Saillance'] == 'nosaliency') & (df['Attaquée'] == 'noattacked')].reset_index()

df_saliency_attacked

,index,Objet,Model,Attaque,Intensité,Saillance,Attaquée,Chemin
0,2,airliner,MobileNetV2,PGD,2,saliency,attacked,c:\Users\lucie\Documents\Programmation\explica...
1,6,airliner,MobileNetV2,PGD,4,saliency,attacked,c:\Users\lucie\Documents\Programmation\explica...
2,10,bear,MobileNetV2,PGD,2,saliency,attacked,c:\Users\lucie\Documents\Programmation\explica...
3,14,bear,MobileNetV2,PGD,4,saliency,attacked,c:\Users\lucie\Documents\Programmation\explica...
4,18,cliffdwelling,MobileNetV2,PGD,2,saliency,attacked,c:\Users\lucie\Documents\Programmation\explica...
5,22,cliffdwelling,MobileNetV2,PGD,4,saliency,attacked,c:\Users\lucie\Documents\Programmation\explica...
6,26,goldfish,MobileNetV2,PGD,2,saliency,attacked,c:\Users\lucie\Documents\Programmation\explica...
7,30,goldfish,MobileNetV2,PGD,4,saliency,attacked,c:\Users\lucie\Documents\Programmation\explica...
8,34,gondola,MobileNetV2,PGD,2,saliency,attacked,c:\Users\lucie\Documents\Programmation\explica...
9,38,gondola,MobileNetV2,PGD,4,saliency,attacked,c:\Users\lucie\Documents\Programmation\explica...


In [6]:
diff_abs_sal_result = []
diff_quadratique_sal_result = []
coef_correlation_sal_result = []
ssim_func_sal_result = []

for i in df_saliency_attacked.index:
    path_sal_map = df_saliency_noattacked['Chemin'][i]
    path_attacked_sal_map = df_saliency_attacked['Chemin'][i]

    # difference metric
    diff_abs_sal_result.append(diff_abs(path_sal_map,path_attacked_sal_map))
    diff_quadratique_sal_result.append(diff_quadratique(path_sal_map,path_attacked_sal_map))
    coef_correlation_sal_result.append(coef_correlation(path_sal_map,path_attacked_sal_map))
    ssim_func_sal_result.append(ssim_func(path_sal_map,path_attacked_sal_map))



In [7]:
diff_abs_img_result = []
diff_quadratique_img_result = []
coef_correlation_img_result = []
ssim_func_img_result = []

for i in df_img_attacked.index:
    path_img_map = df_img_attacked['Chemin'][i]
    path_attacked_img_map = df_img_noattacked['Chemin'][i]

    # difference metric
    diff_abs_img_result.append(diff_abs(path_img_map,path_attacked_img_map))
    diff_quadratique_img_result.append(diff_quadratique(path_img_map,path_attacked_img_map))
    coef_correlation_img_result.append(coef_correlation(path_img_map,path_attacked_img_map))
    ssim_func_img_result.append(ssim_func(path_img_map,path_attacked_img_map))



In [17]:
img_value = pd.DataFrame(list(zip(diff_abs_img_result,diff_quadratique_img_result,coef_correlation_img_result,ssim_func_img_result)), 
                  columns = ['Difference absolue','MSE','Coef de corrélation','SSIM'])

sal_value = pd.DataFrame(list(zip(diff_abs_sal_result,diff_quadratique_sal_result,coef_correlation_sal_result,ssim_func_sal_result)), 
                  columns = ['Difference absolue','MSE','Coef de corrélation','SSIM'])

df_img_info = df_img_attacked.merge(img_value, how='inner', left_index=True, right_index=True)
df_img_info = df_sal_attacked.merge(img_value, how='inner', left_index=True, right_index=True)

,index,Objet,Model,Attaque,Intensité,Saillance,Attaquée,Chemin,Difference absolue,MSE,Coef de corrélation,SSIM
0,0,airliner,MobileNetV2,PGD,2,nosaliency,attacked,c:\Users\lucie\Documents\Programmation\explica...,127.663478,55.433959,0.960599,0.393618
1,4,airliner,MobileNetV2,PGD,4,nosaliency,attacked,c:\Users\lucie\Documents\Programmation\explica...,127.663478,55.433959,0.960599,0.393618
2,8,bear,MobileNetV2,PGD,2,nosaliency,attacked,c:\Users\lucie\Documents\Programmation\explica...,127.641083,63.539787,0.930940,0.517929
3,12,bear,MobileNetV2,PGD,4,nosaliency,attacked,c:\Users\lucie\Documents\Programmation\explica...,127.641083,63.539787,0.930940,0.517929
4,16,cliffdwelling,MobileNetV2,PGD,2,nosaliency,attacked,c:\Users\lucie\Documents\Programmation\explica...,127.531091,55.482987,0.935967,0.756038
5,20,cliffdwelling,MobileNetV2,PGD,4,nosaliency,attacked,c:\Users\lucie\Documents\Programmation\explica...,127.531091,55.482987,0.935967,0.756038
6,24,goldfish,MobileNetV2,PGD,2,nosaliency,attacked,c:\Users\lucie\Documents\Programmation\explica...,126.672127,67.477486,0.911522,0.388965
7,28,goldfish,MobileNetV2,PGD,4,nosaliency,attacked,c:\Users\lucie\Documents\Programmation\explica...,126.672127,67.477486,0.911522,0.388965
8,32,gondola,MobileNetV2,PGD,2,nosaliency,attacked,c:\Users\lucie\Documents\Programmation\explica...,127.932710,56.504517,0.925214,0.520961
9,36,gondola,MobileNetV2,PGD,4,nosaliency,attacked,c:\Users\lucie\Documents\Programmation\explica...,127.932710,56.504517,0.925214,0.520961
